# Unsupervised Learning 2 

## Lesson 3: Market Basket Analysis



In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("small_cust.csv")

In [3]:
df.head()

,CustomerID,Recency,Frequency,Monetary
0,12821,215,6,92.72
1,12823,75,5,1759.50
2,12829,337,11,293.00
3,12831,263,9,215.05
4,12833,146,24,417.38


In [4]:
df.describe()

,CustomerID,Recency,Frequency,Monetary
count,1329.000000,1329.000000,1329.000000,1329.000000
mean,15580.645598,163.078254,14.252069,284.494146
std,1609.275019,101.520251,10.736878,260.984760
min,12821.000000,2.000000,1.000000,3.750000
25%,14176.000000,68.000000,7.000000,142.400000
50%,15567.000000,160.000000,12.000000,223.880000
75%,16988.000000,250.000000,20.000000,344.900000
max,18282.000000,363.000000,77.000000,4055.720000


In [5]:
df.shape

(1329, 4)

In [6]:
sum(df.Frequency)

18941

### Part 2: Getting the transactions

In [7]:
tr = pd.read_csv("C:/Users/Jun Keat/Desktop/AI4I-6 Unsupervised Learning/learnai_unsupervised2/Exercises/OnlineRetail_Filtered.csv")
tr.shape

(339702, 8)

In [8]:
tr[tr.Description == 'POSTAGE'].shape

(27, 8)

In [9]:
tf_filter = tr[tr['CustomerID'].isin(df.CustomerID)]

In [10]:
tf_filter.shape

(18941, 8)

In [11]:
tf_filter['Description'] = tf_filter['Description'].str.strip()
tf_filter['InvoiceNo'] = tf_filter['InvoiceNo'].astype(str)

D:\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [13]:
basket = (tf_filter.groupby(['InvoiceNo','Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo'))

basket.drop('POSTAGE', inplace=True, axis=1)

In [14]:
'POSTAGE' in (basket.columns)

False

In [15]:
basket.head()

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,...,ZINC HEARTS PLANT POT HOLDER,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
538368,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
538369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
538420,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
538470,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
538509,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Part 3: Generating the Association Rules

In [17]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [18]:
def encode_units(x):
     if x <= 0:
         return 0
     if x >= 1:
         return 1

basket_sets = basket.applymap(encode_units)

In [19]:
frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True)

In [22]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(PAINTED METAL PEARS ASSORTED),(ASSORTED COLOUR BIRD ORNAMENT),0.023702,0.050226,0.019187,0.809524,16.117710,0.017997,4.986315
1,(ASSORTED COLOUR BIRD ORNAMENT),(PAINTED METAL PEARS ASSORTED),0.050226,0.023702,0.019187,0.382022,16.117710,0.017997,1.579828
2,(BAKING SET SPACEBOY DESIGN),(BAKING SET 9 PIECE RETROSPOT),0.019752,0.042889,0.015801,0.800000,18.652632,0.014954,4.785553
3,(BAKING SET 9 PIECE RETROSPOT),(BAKING SET SPACEBOY DESIGN),0.042889,0.019752,0.015801,0.368421,18.652632,0.014954,1.552060
4,(BLUE HAPPY BIRTHDAY BUNTING),(PINK HAPPY BIRTHDAY BUNTING),0.015237,0.014108,0.011287,0.740741,52.503704,0.011072,3.802725


In [23]:
rules[(rules['lift'] >= 6) & (rules['confidence'] >= 0.8) ].sort_values(['lift','confidence'], ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
5,(PINK HAPPY BIRTHDAY BUNTING),(BLUE HAPPY BIRTHDAY BUNTING),0.014108,0.015237,0.011287,0.800000,52.503704,0.011072,4.923815
38,"(ROSES REGENCY TEACUP AND SAUCER, PINK REGENCY...",(GREEN REGENCY TEACUP AND SAUCER),0.016366,0.031038,0.014673,0.896552,28.885266,0.014165,9.366629
8,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.024266,0.031038,0.019752,0.813953,26.224101,0.018999,5.208169
33,"(PINK REGENCY TEACUP AND SAUCER, REGENCY CAKES...",(GREEN REGENCY TEACUP AND SAUCER),0.014673,0.031038,0.011851,0.807692,26.022378,0.011396,5.038600
2,(BAKING SET SPACEBOY DESIGN),(BAKING SET 9 PIECE RETROSPOT),0.019752,0.042889,0.015801,0.800000,18.652632,0.014954,4.785553
0,(PAINTED METAL PEARS ASSORTED),(ASSORTED COLOUR BIRD ORNAMENT),0.023702,0.050226,0.019187,0.809524,16.117710,0.017997,4.986315
